# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

[In this notebook you're provided with hints and brief instructions and thinking prompts. Don't ignore them as they are designed to equip you with the structure for the project and will help you analyze what you're doing on a deeper level. Before submitting your project, make sure you remove all hints and descriptions provided to you. Instead, make this report look as if you're sending it to your teammates to demonstrate your findings - they shouldn't know you had some external help from us! To help you out, we've placed the hints you should remove in square brackets.]

[Before you dive into analyzing your data, explain the purposes of the project and hypotheses you're going to test.]

## Open the data file and have a look at the general information. 

[Start with importing the libraries and loading the data. You may realise that you need additional libraries as you go, which is totally fine - just make sure to update this section when you do.]

In [1]:
# Loading all the libraries
import pandas as pd

df = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

[Now let's explore our data. You'll want to see how many columns and rows it has, look at a few rows to check for potential issues with the data.]

In [32]:
# Let's see how many rows and columns our dataset has
df.shape

(21525, 12)

In [33]:
# let's print the first N rows
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


[Describe what you see and notice in your printed data sample. Are there any issues that may need further investigation and changes?]
Issues I consider investigating:
1. Days_Employed column: I will make all posotives because you can't work negative days
2. Education column: I will make all lowercase for formatting 

In [2]:
df['days_employed'] = df['days_employed'].abs()
df['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [35]:
df['education'].apply(str)

0          bachelor's degree
1        secondary education
2        Secondary Education
3        secondary education
4        secondary education
                ...         
21520    secondary education
21521    secondary education
21522    secondary education
21523    secondary education
21524    secondary education
Name: education, Length: 21525, dtype: object

In [36]:
# Get info on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


[Are there missing values across all columns or just a few? Briefly describe what you see in 1-2 sentences.]

2/12 columns are missing values. The dataset has 21525 entries. The 'days_employed', 'total_income' columns are less than 21525. This means those columns are missing values. 

In [37]:
# Let's look at the filtered table with missing values in the the first column with missing data
df['days_employed'].isna().sum()

2174

[Do missing values seem symmetric? Can we be sure in this assumption? Explain your thoughts briefly in this section. You may probably want to conduct further investigations, and count the missing values in all the rows with missing values to confirm the the missing samples are of the same size.]

They are symmetric bc they both say 19351. The DataFrame has 21525 enteries, minus 19351, equals 2174 missing values for total_income and days_employed.  

In [38]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.
filtered_table_days_employed = df['days_employed'].value_counts(dropna=False)
filtered_table_total_income = df['total_income'].value_counts(dropna=False)

**Intermediate conclusion**

[Does the number of rows in the filtered table match the number of missing values? What conclusion can we make from this?] 
The number of rows DOES match the filtered table number of missing values. I conclude the missing values are symmetric of the missing sample size

[Calculate the percentage of the missing values compared to the whole dataset. Is it a considerably large piece of data? If so, you may want to fill the missing values. To do that, firstly we should consider whether the missing data could be due to the specific client characteristic, such as employment type or something else. You will need to decide which characteristic *you* think might be the reason. Secondly, we should check whether there's any dependence missing values have on the value of other indicators with the columns with identified specific client characteristic.]
Divide 19351 by 21525 and multiply that by 100 - you get roughly 89.9% 
Professionally I prefer to be closer to 100%. In my opinion, the missing data is a result of unclear information for the developer of the data. 

[Explain your next steps and how they correlate with the conclusions you made so far.]

In [39]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [3]:
# Checking distribution
df['days_employed'].value_counts(dropna=False)
df['total_income'].value_counts(dropna=False)

NaN          2174
17312.717       2
31791.384       2
42413.096       2
22435.069       1
             ... 
23834.534       1
26124.613       1
28692.182       1
28477.783       1
41428.916       1
Name: total_income, Length: 19349, dtype: int64

[Describe your findings here.]
**Possible reasons for missing values in data** 
'I observe that days_employed and total_income both have 2174 missing values.'
'I consider the possibility that 2174 people chose not to provide this information'

[Propose your ideas on why you think the values might be missing. Do you think they are missing randomly or there are any patterns?]
'I use the logic that a person might not have income if they are not employed'
'With this logic I conclude this was not random, but intentional'

[Let's start checking why the missing values are not random.]

In [41]:
# Checking the distribution in the whole dataset
for x in df.columns:
    print(x)
    print(df[x].value_counts(normalize=True))

children
 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64
days_employed
986.927316     0.000052
7026.359174    0.000052
4236.274243    0.000052
6620.396473    0.000052
1238.560080    0.000052
                 ...   
2849.351119    0.000052
5619.328204    0.000052
448.829898     0.000052
1687.038672    0.000052
582.538413     0.000052
Name: days_employed, Length: 19351, dtype: float64
dob_years
35    0.028664
40    0.028293
41    0.028200
34    0.028014
38    0.027782
42    0.027735
33    0.026992
39    0.026620
31    0.026016
36    0.025784
44    0.025412
29    0.025319
30    0.025087
48    0.024994
37    0.024948
50    0.023879
43    0.023833
32    0.023693
49    0.023600
28    0.023368
45    0.023089
27    0.022904
56    0.022625
52    0.022485
47    0.022300
54    0.022253
46    0.022067
58    0.021417
57    0.021370
53    0.021324
51    0.020813
59    0.020627
55    0.02058

**Intermediate conclusion**

[Is the distribution in the original dataset similar to the distribution of the filtered table? What does that mean for us?]

[If you think we can't make any conclusions yet, let's investigate our dataset further. Let's think about other reasons that could lead to data missing and check if we can find any patterns that may lead us to thinking that the missing values are not random. Because this is your work, this is section is optional.]

In [4]:
#print unique values
visits = df.groupby('gender')['total_income'].value_counts(normalize=True)
visits.unique()

array([7.84190715e-05, 1.51561079e-04, 1.00000000e+00])

**Intermediate conclusion**

[Can we finally confirm that missing values are accidental? Check for anything else that you think might be important here.]

In [5]:
df.loc[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [44]:
# Check for other reasons and patterns that could lead to missing values
df.loc[df['total_income'].isna()].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.552438,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.469356,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


Checking for other patterns - explain which

I conclude the pattern is established above. 
I conclude that days_employted and total_income have missing values for the same clients.

**Conclusions**

[Did you find any patterns? How did you come to this conclusion?]
I oberseve a count of 19348 and 19348 unique values. 
[Explain how you will address the missing values. Consider the categories in which values are missing.]
'I believe the best thing would be to change the missing values to None.'
[Briefly plan your next steps for transforming data. You will probably need to address different types of issues: duplicates, different registers, incorrect artifacts, and missing values.]
I need to fix education, days_employed, 

## Data transformation

[Let's go through each column to see what issues we may have in them.]

[Begin with removing duplicates and fixing educational information if required.]

In [45]:
# Let's see all values in education column to check if and what spellings will need to be fixed
df['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64

In [46]:
# Fix the registers if required
df['education'] = df['education'].str.lower()
df['education']

0          bachelor's degree
1        secondary education
2        secondary education
3        secondary education
4        secondary education
                ...         
21520    secondary education
21521    secondary education
21522    secondary education
21523    secondary education
21524    secondary education
Name: education, Length: 21525, dtype: object

In [47]:
df['days_employed'] = df['days_employed'].abs()

In [48]:
# Checking all the values in the column to make sure we fixed them
df['education'].value_counts(),df['days_employed'].value_counts()

(secondary education    15233
 bachelor's degree       5260
 some college             744
 primary education        282
 graduate degree            6
 Name: education, dtype: int64,
 986.927316     1
 7026.359174    1
 4236.274243    1
 6620.396473    1
 1238.560080    1
               ..
 2849.351119    1
 5619.328204    1
 448.829898     1
 1687.038672    1
 582.538413     1
 Name: days_employed, Length: 19351, dtype: int64)

[Check the data the `children` column]

In [49]:
# Let's see the distribution of values in the `children` column
print(df['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


[Are there any strange things in the column? If yes, how high is the percentage of problematic data? How could they have occurred? Make a decision on what you will do with this data and explain you reasoning.]

I observe '-1' as an illogical value for number of children. This accounts for (47/21525)*100 = 0.22% of the data. I will assume the negative symbol was not intentional. Therefore, adding the enteries for -1 children to the 1 child column is my choice. 

In [7]:
# Checking the `children` column again to make sure it's all fixed
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

[Check the data in the `days_employed` column. Firstly think about what kind of issues could there be and what you may want to check and how you will do it.]

In [52]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
df['days_employed'].value_counts()

986.927316     1
7026.359174    1
4236.274243    1
6620.396473    1
1238.560080    1
              ..
2849.351119    1
5619.328204    1
448.829898     1
1687.038672    1
582.538413     1
Name: days_employed, Length: 19351, dtype: int64

[If the amount of problematic data is high, it could've been due to some technical issues. We may probably want to propose the most obvious reason why it could've happened and what the correct data might've been, as we can't drop these problematic rows.]

In [53]:
# Address the problematic values, if they exist
df['days_employed'] = df['days_employed'].abs()
df['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [54]:
# Check the result - make sure it's fixed
df['days_employed'].value_counts()

986.927316     1
7026.359174    1
4236.274243    1
6620.396473    1
1238.560080    1
              ..
2849.351119    1
5619.328204    1
448.829898     1
1687.038672    1
582.538413     1
Name: days_employed, Length: 19351, dtype: int64

[Let's now look at the client's age and whether there are any issues there. Again, think about what can data can be strange in this column, i.e. what cannot be someone's age.]

In [55]:
# Check the `dob_years` for suspicious values and count the percentage
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

[Decide what you'll do with the problematic values and explain why.]

It's illogical for this column to have '0' as a value for age. I assume this means the person filling out the form left it blank. I will convert the value of '0' to NaN.

In [56]:
# Address the issues in the `dob_years` column, if they exist
df['dob_years'] = df['dob_years'].isna()

In [57]:
# Check the result - make sure it's fixed
df['dob_years'].value_counts()

False    21525
Name: dob_years, dtype: int64

[Now let's check the `family_status` column. See what kind of values there are and what problems you may need to address.]

In [58]:
# Let's see the values for the column
df['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [59]:
# Address the problematic values in `family_status`, if they exist
print ('nothing wrong with family_status')

nothing wrong with family_status


In [60]:
# Check the result - make sure it's fixed
print ('nothing wrong with family_status')

nothing wrong with family_status


[Now let's check the `gender` column. See what kind of values there are and what problems you may need to address]

In [61]:
# Let's see the values in the column
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [62]:
# Address the problematic values, if they exist
df = df.loc[df['gender']!='XNA']
df.head()
#df['gender'].drop(['XNA',axis = 0], inplace=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,False,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,False,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,False,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,False,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,False,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [63]:
# Check the result - make sure it's fixed

df['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

[Now let's check the `income_type` column. See what kind of values there are and what problems you may need to address]

In [64]:
# Let's see the values in the column
df['income_type'].value_counts()

employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [65]:
# Address the problematic values, if they exist
# -1 children is the problematic value so I will remove it
for x in df.columns:
    print(x)
    print(df[x].value_counts(normalize=True))

children
 0     0.657313
 1     0.223843
 2     0.095475
 3     0.015332
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64
days_employed
986.927316     0.000052
7026.359174    0.000052
4236.274243    0.000052
6620.396473    0.000052
1238.560080    0.000052
                 ...   
5619.328204    0.000052
448.829898     0.000052
1687.038672    0.000052
2348.524271    0.000052
582.538413     0.000052
Name: days_employed, Length: 19350, dtype: float64
dob_years
False    1.0
Name: dob_years, dtype: float64
education
secondary education    0.707722
bachelor's degree      0.244378
some college           0.034520
primary education      0.013102
graduate degree        0.000279
Name: education, dtype: float64
education_id
1    0.707722
0    0.244378
2    0.034520
3    0.013102
4    0.000279
Name: education_id, dtype: float64
family_status
married              0.575172
civil partnership    0.194016
unmarried            0.130691
divorced             0.0

In [66]:
#removing -1 in children
df = df.loc[(df['children']!= -1)&(df['children']!= 20)]
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,False,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,False,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,False,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,False,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,False,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


we removed gender_debt 

In [67]:
# Check the result - make sure it's fixed
df['children'].value_counts()


0    14148
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

[Now let's see if we have any duplicates in our data. If we do, you'll need to decide what you will do with them and explain why.]

In [68]:
# Checking duplicates

df.duplicated().sum()

768

In [69]:
# Address the duplicates, if they exist
df = df.drop_duplicates().reset_index(drop=True)

In [71]:
# Last check whether we have any duplicates
df.duplicated().sum()

0

In [72]:
# Check the size of the dataset that you now have after your first manipulations with it
df.shape, df.columns

((20633, 12),
 Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
        'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
        'total_income', 'purpose'],
       dtype='object'))

[Describe your new dataset: briefly say what's changed and what's the percentage of the changes, if there were any.]


Checked for duplicates, removed duplicates, number of rows decreased by 4.14% in dataframe. 

Also new column for Index

In [73]:
(20633/21525)*100

95.85598141695702

In [74]:
100-95.85598141695702

4.144018583042978


# Working with missing values

[To speed up working with some data, you may want to work with dictionaries for some values, where IDs are provided. Explain why and which dictionaries you will work with.]

In [75]:
# Find the dictionaries
df.columns.unique()

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

### Restoring missing values in `total_income`

[Briefly state which column(s) have values missing that you need to address. Explain how you will fix them.]


[Start with addressing total income missing values. Create and age category for clients. Create a new column with the age category. This strategy can help with calculating values for the total income.]


In [76]:
# Let's write a function that calculates the age category

print("The client is : ")

def age_category(dob_years):
    if dob_years >= 20 and dob_years <= 35:
        return 'Young Adult'
    elif dob_years >= 36 and dob_years <= 50:
        return 'Middle Aged'
    elif dob_years >= 51 and dob_years <=75:
        return 'Senior'
    else:
        return 'Not in DataFrame'

The client is : 


In [77]:
# Test if the function works and create new column
print('Call Function and Create Column : ')

df['age_category'] = df['dob_years'].apply(age_category)
print(age_category(27))
print(age_category(17))
print(age_category(66))

Call Function and Create Column : 
Young Adult
Not in DataFrame
Senior


In [78]:
# Check new column values based on the function we created
print('Check New Column')
df['age_category'].value_counts()
#df['new_age_value'] = pd.DataFrame(df['dob_years'])

Check New Column


Not in DataFrame    20633
Name: age_category, dtype: int64

[Think about the factors on which income usually depends. Eventually, you will want to find out whether you should use mean or median values for replacing missing values. To make this decision you will probably want to look at the distribution of the factors you identified as impacting one's income.]

[Create a table that only has data without missing values. This data will be used to restore the missing values.]

In [79]:
# Create a table without missing values and print a few of its rows to make sure it looks fine
without_missing_values = df[df.notna().all(axis=1)]
without_missing_values.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_category        0
dtype: int64

In [80]:
# Look at the mean values for income based on your identified factors
mean_income = without_missing_values['total_income'].mean()
print(mean_income)

26791.503175061076


In [81]:
# Look at the median values for income based on your identified factors
median_income = without_missing_values['total_income'].median()
print(median_income)

23202.87


In [83]:
df['total_income'].describe()

count     19239.000000
mean      26791.503175
std       16497.194890
min        3306.762000
25%       16479.999500
50%       23202.870000
75%       32549.611000
max      362496.645000
Name: total_income, dtype: float64

[Repeat such comparisons for multiple factors. Make sure you consider different aspects and explain your thinking process.]
median because it isn't manipulated by large values.The mean is used for normal distributions, because it's influnced by outliers



[Make a decision on what characteristics define income most and whether you will use a median or a mean. Explain why you made this decision]


In [84]:
#  Write a function that we will use for filling in missing values


def fill_missing_values(fill_col, fill_value, cat_1, cat_2):
    df[fill_col] = df[fill_col].fillna(df.groupby([cat_1 , cat_2])[fill_col].transform(fill_value))
    return df[fill_col].isna().sum()        
       

In [85]:
# Check if it works
# Apply it to every row
fill_missing_values(fill_col='total_income', fill_value='median', cat_1='gender', cat_2='debt') 

0

In [86]:
# Check if we got any errors
check_df=pd.read_csv('/datasets/credit_scoring_eng.csv')
check_df.groupby(['gender','debt'])['total_income'].median()

gender  debt
F       0       21470.354
        1       21346.304
M       0       27099.601
        1       25518.842
XNA     0       32624.825
Name: total_income, dtype: float64

We filled missing values with gender and debt. Used .loc() to fill in missing values.

In [87]:
df.loc[(df['gender']=='F') & (df['debt']==0) & (df['total_income'] ==21470.354)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
7036,0,2567.889652,False,secondary education,1,civil partnership,1,F,civil servant,0,21470.354,wedding ceremony,Not in DataFrame


[If you've came across errors in preparing the values for missing data, it probably means there's something special about the data for the category. Give it some thought - you may want to fix some things manually, if there's enough data to find medians/means.]


In [88]:
# Replacing missing values if there are any errors
df['total_income'].value_counts()



21467.9410    815
27103.4160    423
21248.2895     94
25518.8420     63
31791.3840      2
             ... 
19232.3340      1
32804.1990      1
16773.4570      1
9762.8240       1
9591.8240       1
Name: total_income, Length: 19239, dtype: int64

[When you think you've finished with `total_income`, check that the total number of values in this column matches the number of values in other ones.]

In [89]:
# No errors found
df['total_income'].nunique()

19239

In [90]:
# Checking the number of entries in the columns
df['education'].count() , df['total_income'].count()

(20633, 20633)

###  Restoring values in `days_employed`

[Think about the parameters that may help you restore the missing values in this column. Eventually, you will want to find out whether you should use mean or median values for replacing missing values. You will probably conduct a research similar to the one you've done when restoring data in a previous column.]

Days_employed was converted to absolute values above

In [91]:
df['days_employed'].value_counts()

986.927316       1
1893.222792      1
1741.489608      1
5135.928528      1
1453.358707      1
                ..
2348.524271      1
2338.480708      1
356642.853685    1
1218.820922      1
582.538413       1
Name: days_employed, Length: 19239, dtype: int64

In [92]:
# Distribution of `days_employed` medians based on your identified parameters
median_days_employed = df.groupby(['gender','debt'])['days_employed'].median().reset_index()

median_days_employed['days_employed'].value_counts()
#median_days_employed['days_employed'].nunique()


1856.789482    1
1201.819224    1
2611.551298    1
1734.469610    1
Name: days_employed, dtype: int64

In [93]:
# Distribution of `days_employed` means based on your identified parameters
mean_days_employed = df.groupby(['gender','debt'])['days_employed'].mean().reset_index()

mean_days_employed['days_employed'].value_counts()

38436.685162    1
58517.596432    1
84344.551918    1
25250.677349    1
Name: days_employed, dtype: int64

[Decide what you will use: means or medians. Explain why.]
mean is for normal distributions, median is skewed distributions. The mean is largely influenced by outliers. The median is much more robust and sensible.

In [94]:
# Let's write a function that calculates means or medians (depending on your decision) based on your identified parameter
def calculate_value(fill_col, fill_value, cat_1, cat_2):
    df[fill_col] = df[fill_col].fillna(df.groupby([cat_1 , cat_2])[fill_col].transform(fill_value))
    return df[fill_col].value_counts()


In [95]:
# Check that the function works
# Apply function to the income_type

calculate_value('days_employed','median','debt', 'income_type')

1617.008140      567
1577.876626      347
365360.781158    200
2698.812988      124
1204.008348       91
                ... 
1687.038672        1
2348.524271        1
2338.480708        1
356642.853685      1
582.538413         1
Name: days_employed, Length: 19243, dtype: int64

In [96]:
# Check if function worked
#The function worked
df['income_type'].value_counts()

employee                       10606
business                        4924
retiree                         3659
civil servant                   1438
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [97]:
# Replacing missing values

df['income_type'].isna().sum()

0

[When you think you've finished with `total_income`, check that the total number of values in this column matches the number of values in other ones.]

In [98]:
# Check the entries in all columns - make sure we fixed all missing values
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_category        0
dtype: int64

## Categorization of data

[To answer the questions and test the hypotheses, you will want to work with categorized data. Look at the questions that were posed to you and that you should answer. Think about which of the data will need to be categorized to answer these questions. Below you will find a template through which you can work your way when categorizing data. The first step-by-step processing covers the text data; the second one addresses the numerical data that needs to be categorized. You can use both or none of the suggested instructions - it's up to you.]

[Despite of how you decide to address the categorization, make sure to provide clear explanation of why you made your decision. Remember: this is your work and you make all decisions in it.]


In [99]:
# Print the values for your selected data for categorization
# Check the unique values
for value_counts in df.columns:
    print(value_counts)
    print(df[value_counts].unique())  
    print('-'*40)


children
[1 0 3 2 4 5]
----------------------------------------
days_employed
[8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]
----------------------------------------
dob_years
[False]
----------------------------------------
education
["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']
----------------------------------------
education_id
[0 1 2 3 4]
----------------------------------------
family_status
['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']
----------------------------------------
family_status_id
[0 1 2 3 4]
----------------------------------------
gender
['F' 'M']
----------------------------------------
income_type
['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']
----------------------------------------
debt
[0 1]
----------------------------------------
total_income
[40620.102 17932.802 233

[Let's check unique values]

[What main groups can you identify based on the unique values?]

[Based on these themes, we will probably want to categorize our data.]


In [100]:
# Let's write a function to categorize the data based on common topics
def eligibility(debt,gender):
    if debt == 1 and gender == 'M':
        return 'Not Elligable'
    elif debt == 0 and gender == 'M':
        return 'Elligable'
    elif debt == 1 and gender == 'F':
        return 'Pending'
    else:
        return 'Pending'

In [101]:
# Create a column with the categories and count the values for them


#df['elligability'] = df[['debt' & 'gender']].apply(lambda x: gender_debt(x))

df_eligibility = df.groupby(['age_category', 'income_type']) \
    .agg(persons=('debt', 'count'), 
         in_group_debt_rate=('debt', 'mean'),
         population_debt_rate=('debt', 'sum'
                              
                              ))
#lambda x:
#sum(x)/len(df)
#df['debt'] = df['gender'].apply(eligibility)

[If you decide to categorize the numerical data, you'll need to come up with the categories for it too.]

In [102]:
# Looking through all the numerical data in your selected column for categorization
for value_counts in df_eligibility.columns:
    print(value_counts, '\n' )
    print(df_eligibility[value_counts].unique())  
    print('-'*40)

persons 

[ 4924  1438 10606     2     1  3659]
----------------------------------------
in_group_debt_rate 

[0.07575142 0.05980529 0.09871771 0.         1.         0.05766603
 0.5       ]
----------------------------------------
population_debt_rate 

[ 373   86 1047    0    1  211]
----------------------------------------


In [103]:
# Getting summary statistics for the column
df_eligibility.describe()


,persons,in_group_debt_rate,population_debt_rate
count,8.000000,8.000000,8.000000
mean,2579.125000,0.223993,214.875000
std,3760.206318,0.352745,362.114204
min,1.000000,0.000000,0.000000
25%,1.750000,0.043250,0.750000
50%,720.000000,0.067778,43.500000
75%,3975.250000,0.199038,251.500000
max,10606.000000,1.000000,1047.000000


[Decide what ranges you will use for grouping and explain why.] I used these ranges based on .describe() method I used on days employed. This was done to acheive proper distribution over the dataframe

In [104]:
# Creating function for categorizing into different numerical groups based on ranges
#def ranges (debt,gender): 
    #if 1 == debt & 'F' == gender:
        #return "Female and On Time"
    #if 1 == debt & 'M' == gender:
        #return "Male and On Time"
    #if 0 == debt & 'F' == gender:
        #return "Female with Debt"
    #if 0 == debt & 'M' == gender:
        #return "Male with Debt"
        #else
            #return "Not in our DataFrame"
            
def ranges(row):
    days_employed = row['days_employed']
    total_income = row['total_income']
    if days_employed >= 2747 & total_income >= 16488:
        return '25 percentile'
    if days_employed >= 1203 & total_income >= 23202:
        return '50 percentile'
    if days_employed >= 291 & total_income >= 32549:
        return '75 percentile'
    else:
        return 'Outlier'
    

In [105]:
df['days_employed'].describe()

count     20633.000000
mean      66667.200987
std      138857.826381
min          24.141633
25%         994.321333
50%        2060.164064
75%        5367.877964
max      401755.400475
Name: days_employed, dtype: float64

In [106]:
df['total_income'].describe()

count     20633.000000
mean      26558.736237
std       15968.093364
min        3306.762000
25%       16944.193000
50%       22912.349000
75%       31713.445000
max      362496.645000
Name: total_income, dtype: float64

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [109]:
# Check the children data and paying back on time
dataframe_children=df[['children','debt']]
dataframe_children.head()






# def payback_rate(debt,children): 
#     if debt==1 and children >0:
#         return "Debt with Kids"
#     elif debt==1 and children ==0:
#         return "Debt no Kids"
#     elif debt==0 and children >0:
#         return "No Debt with Kids"
#     elif debt==0 and children <0:
#         return "No Debt no Kids"
#     else:
#         return "Not in our DataFrame"




df_eligibility = df.groupby(['age_category', 'income_type']) \
    .agg(persons=('debt', 'count'), 
         in_group_debt_rate=('debt', 'mean'),
         population_debt_rate=('debt', 'sum'
                              
                              ))

In [110]:
# Calculating default-rate based on the number of children
df_child_ratio = df.groupby(['children'])\
    .agg(persons=('debt', 'sum'), 
         debt_ratio=('debt', 'mean'))
df_child_ratio.head()

,persons,debt_ratio
children,,
0,1051,0.077565
1,443,0.094456
2,194,0.096326
3,27,0.082067
4,4,0.097561


**Conclusion**

[Write your conclusions based on your manipulations and observations.] I observe a correlation between children and debt. The debt ratio is the lowest with no kids,and the highest with 4 kids. The debt ratio for three kids stands out because its lower than 1 or 2 kids.


**Is there a correlation between family status and paying back on time?**

In [111]:
# Check the family status data and paying back on time

df.groupby(['family_status'])\
    .agg(persons=('debt', 'sum'), 
         debt_ratio=('debt', 'mean'))

# Calculating default-rate based on family status
#df['family_status']


,persons,debt_ratio
family_status,,
civil partnership,378,0.094808
divorced,84,0.071429
married,921,0.078110
unmarried,273,0.099381
widow / widower,63,0.067597


**Conclusion**

[Write your conclusions based on your manipulations and observations.] The debt ratio is highest when unmarried, and lowest as widow/widower. Divorced and married people have similar debt ratio. 

**Is there a correlation between income level and paying back on time?**

In [115]:
# Check the income level data and paying back on time

df.groupby(['income_type'])\
    .agg(persons=('debt', 'sum'), 
         debt_ratio=('debt', 'mean'))


# Calculating default-rate based on income level



,persons,debt_ratio
income_type,,
business,373,0.075751
civil servant,86,0.059805
employee,1047,0.098718
entrepreneur,0,0.000000
paternity / maternity leave,1,1.000000
retiree,211,0.057666
student,0,0.000000
unemployed,1,0.500000


**Conclusion**

[Write your conclusions based on your manipulations and observations.]
I did not observe  a correlation between income type and debt ratio.

**How does credit purpose affect the default rate?**

In [113]:
# Check the percentages for default rate for each credit purpose and analyze them

df.groupby(['purpose'])\
    .agg(persons=('debt', 'sum'), 
         debt_ratio=('debt', 'mean'))

,persons,debt_ratio
purpose,,
building a property,54,0.090000
building a real estate,48,0.079470
buy commercial real estate,46,0.072214
buy real estate,43,0.073129
buy residential real estate,41,0.071057
buying a second-hand car,35,0.076253
buying my own car,46,0.094845
buying property for renting out,51,0.081731
car,41,0.085062
